## [Baseline] 1. 누구나 할 수 있는 데이터 분석 & 예측 따라해보기

### 1. 데이터 불러오기

In [4]:
import pandas as pd

train = pd.read_csv('train.csv')

train.head(3)

,id,Species,Island,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Body Mass (g)
0,0,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,50.0,15.3,220,MALE,8.30515,-25.19017,5550
1,1,Chinstrap penguin (Pygoscelis antarctica),Dream,No,49.5,19.0,200,MALE,9.63074,-24.34684,3800
2,2,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,45.1,14.4,210,FEMALE,8.51951,-27.01854,4400


---

### 2. 결측치 확인

In [6]:
def check_missing_col(dataframe):
    missing_col = []
    counted_missing_col = 0
    for i, col in enumerate(dataframe.columns):
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            counted_missing_col += 1
            print(f'결측치가 있는 컬럼은: {col}입니다.')
            print(f'해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if counted_missing_col == 0:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(train)

결측치가 있는 컬럼은: Sex입니다.
해당 컬럼에 총 3개의 결측치가 존재합니다.
결측치가 있는 컬럼은: Delta 15 N (o/oo)입니다.
해당 컬럼에 총 3개의 결측치가 존재합니다.
결측치가 있는 컬럼은: Delta 13 C (o/oo)입니다.
해당 컬럼에 총 3개의 결측치가 존재합니다.


---

### 3. 결측치 처리

In [8]:
train[train.isna().sum(axis=1) > 0]

,id,Species,Island,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Body Mass (g)
6,6,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,42.0,20.2,190,NaN,9.13362,-25.09368,4250
8,8,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,34.1,18.1,193,NaN,NaN,NaN,3475
18,18,Adelie Penguin (Pygoscelis adeliae),Dream,No,39.8,19.1,184,MALE,NaN,NaN,4650
70,70,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,46.2,14.4,214,NaN,8.24253,-26.81540,4650
109,109,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,36.6,17.8,185,FEMALE,NaN,NaN,3700


In [13]:
# 결측치를 처리하는 함수_이해 x
def handle_na(data, missing_col):
    temp = data.copy()
    for col, dtype in missing_col:
        if dtype == 'O':
            temp = temp.dropna(subset = [col])
        elif dtype == int or dtype == float:
            temp.loc[:,col] = temp[col].fillna(temp[col].mean())
    return temp

data = handle_na(train, missing_col)

# 결측치 처리 확인
missing_col = check_missing_col(train)

결측치가 있는 컬럼은: Sex입니다.
해당 컬럼에 총 3개의 결측치가 존재합니다.
결측치가 있는 컬럼은: Delta 15 N (o/oo)입니다.
해당 컬럼에 총 3개의 결측치가 존재합니다.
결측치가 있는 컬럼은: Delta 13 C (o/oo)입니다.
해당 컬럼에 총 3개의 결측치가 존재합니다.


---

### 4. 데이터 설명 방정식 만들기 (Y = aX + b)

* train.csv를 이용해서 데이터 사이의 관계를 파악
* 펭귄의 날개 길이(X)를 이용해서 펭귄의 몸무게(Y) 예측  
cf. corr을 통해 상관관계를 확인해보면 변수 요인파악이 가능

In [26]:
train.corr()
# 몸무게(Y)에 영향을 미치는 가장 큰 요인으로 날개 길이(X)를 꼽을 수 있다

,id,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Delta 15 N (o/oo),Delta 13 C (o/oo),Body Mass (g)
id,1.000000,0.078787,-0.144788,0.077909,-0.035484,0.024416,0.089186
Culmen Length (mm),0.078787,1.000000,-0.251255,0.672205,-0.040205,0.114983,0.572063
Culmen Depth (mm),-0.144788,-0.251255,1.000000,-0.603341,0.650629,0.477688,-0.490643
Flipper Length (mm),0.077909,0.672205,-0.603341,1.000000,-0.533212,-0.454315,0.864814
Delta 15 N (o/oo),-0.035484,-0.040205,0.650629,-0.533212,1.000000,0.641107,-0.548678
Delta 13 C (o/oo),0.024416,0.114983,0.477688,-0.454315,0.641107,1.000000,-0.468425
Body Mass (g),0.089186,0.572063,-0.490643,0.864814,-0.548678,-0.468425,1.000000


In [19]:
X = train['Flipper Length (mm)']
Y = train['Body Mass (g)']

---

### 5. a 구하기

In [29]:
import numpy as np

# X와 Y의 평균을 구함
mean_X = np.mean(X)
mean_Y = np.mean(Y)

# mean_Y를 mean_X로 나누어서 두 변수간 관계를 확인
print(mean_Y/mean_X)

# 몸무게는 날개 길이의 21.31배 정도 임을 파악할 수 있다
# Y = 21.31 X + b 식을 세울 수 있다

21.31393640919302


---

### 6. b 구하기

In [31]:
# b = Y - 21.31 X 식을 그대로 사용, 평균값을 그대로 사용한다
b = mean_Y - 21.31 * mean_X
b

# b는 0.8 이다

0.7992982456144091

---

### 7. 방정식 Y = 21.31 X + 0.8 이용해서 몸무게 예측

In [35]:
predict_Y = 21.31 * X + 0.8
predict_Y

0      4689.00
1      4262.80
2      4475.90
3      4561.14
4      4795.55
        ...   
109    3943.15
110    4049.70
111    4092.32
112    4092.32
113    4859.48
Name: Flipper Length (mm), Length: 114, dtype: float64

---

### 8. 예측 결과 평가하기
더 정확한 예측을 위해 개선점 확인
* 대회의 평가지표는 RMSE(Root Mean of Squared Error)
* 예측 값과 실제 값 사이, 차이값들의 제곱값들의 평균 (양수, 음수 고려)

In [39]:
# 대회 규칙의 평가 산식 함수를 그대로 사용 (cf. 대회 규칙 - 평가 )
import numpy as np

def RMSE(true, pred):
    score = np.sqrt(np.mean(np.square(true-predict_Y)))
    return score

# 실제 정답 값 준비
real_answer = Y.copy() 

# 정답값과 예측값의 함수를 넣어 결과 확인
error = RMSE(real_answer, predict_Y)

print(error)

# 평균 에러는 532.84

532.843391763109


---

### 9. test_data 예측하기

In [40]:
# 예측하고자 하는 test_data 불러오기
test = pd.read_csv('test.csv')
test.head()

# Body Mass(g)가 존재하지 않는 것을 확인할 수 있다
# 방정식을 이용해서 test 데이터에 Body Mass(g)를 채워볼 것이다

,id,Species,Island,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo)
0,0,Chinstrap penguin (Pygoscelis antarctica),Dream,Yes,52.0,20.7,210.0,MALE,9.43146,-24.68440
1,1,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,55.9,17.0,228.0,MALE,8.31180,-26.35425
2,2,Adelie Penguin (Pygoscelis adeliae),Dream,Yes,38.9,18.8,190.0,FEMALE,8.36936,-26.11199
3,3,Chinstrap penguin (Pygoscelis antarctica),Dream,Yes,45.2,16.6,191.0,FEMALE,9.62357,-24.78984
4,4,Adelie Penguin (Pygoscelis adeliae),Biscoe,No,37.9,18.6,172.0,FEMALE,8.38404,-25.19837


In [41]:
# 예측의 재료 가져오기
test_X = test['Flipper Length (mm)']

# 예측 진행
predict_test = 21.31 * test_X + 0.8

# 예측 결과 확인
predict_test

0      4475.90
1      4859.48
2      4049.70
3      4071.01
4      3666.12
        ...   
223    4326.73
224    4625.07
225    4475.90
226    4284.11
227    4134.94
Name: Flipper Length (mm), Length: 228, dtype: float64

---

### 10. 대회 제출을 위한 파일 저장

In [45]:
submission = pd.read_csv('sample_submission.csv')
submission.head()

,id,Body Mass (g)
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [46]:
submission['Body Mass (g)'] = predict_test

submission

,id,Body Mass (g)
0,0,4475.90
1,1,4859.48
2,2,4049.70
3,3,4071.01
4,4,3666.12
...,...,...
223,223,4326.73
224,224,4625.07
225,225,4475.90
226,226,4284.11


In [48]:
# submission 파일 csv로 저장
# index=False : 추가적인 id를 부여할 필요가 없다 (정확한 채점을 위해 필요한 옵션)
submission.to_csv('sumission.csv', index=False)